In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [2]:
MODEL_GPT = 'gpt-4o-mini'

## Basic App: Question & Answering from Document

In [3]:
# from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI

In [4]:
# llm = OpenAI()
llm = ChatOpenAI(model=MODEL_GPT)

### Load text file

In [5]:
from langchain.document_loaders import TextLoader

In [6]:
# loader = TextLoader("data/be-good-and-how-not-to-die.txt")
loader = TextLoader("../../data/be-good-and-how-not-to-die.txt")

In [7]:
document = loader.load()

### Document is loaded as Python list with metadata

In [8]:
print(type(document))

<class 'list'>


In [9]:
print(len(document))

1


In [10]:
print(document[0].metadata)

{'source': '../../data/be-good-and-how-not-to-die.txt'}


In [11]:
print(f"You have {len(document)} document.")

You have 1 document.


In [12]:
print(f"Your document has {len(document[0].page_content)} characters")

Your document has 27423 characters


### Split document in small chunks

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [14]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000,
    chunk_overlap=400
)

In [15]:
document_chunks = text_splitter.split_documents(document)

In [16]:
print(f"Now you have {len(document_chunks)} chunks.")

Now you have 12 chunks.


### Convert text chunks in numeric vectors (called "embeddings")

In [17]:
# from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings

In [18]:
embeddings = OpenAIEmbeddings()

### Load embeddings to vector database

In [19]:
from langchain.vectorstores import FAISS

In [20]:
stored_embeddings = FAISS.from_documents(document_chunks, embeddings)

### Create Retrieval Question & Answering Chain

In [21]:
from langchain.chains import RetrievalQA

In [22]:
QA_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=stored_embeddings.as_retriever()
)

### Now we have Question & Answering App

In [23]:
question = """
What is this article about? 
Describe it in less than 100 words.
"""

In [24]:
# QA_chain.run(question)
QA_chain.invoke(question)

{'query': '\nWhat is this article about? \nDescribe it in less than 100 words.\n',
 'result': 'The article discusses the principles of startup success, emphasizing the importance of creating products that people want while not overly focusing on immediate monetization. It suggests that startups should operate with a mindset similar to charities, prioritizing user satisfaction. The author shares insights from personal experiences and examples like Craigslist and Google, highlighting that a significant number of startups may fail but persistence and regular communication are key to survival. Ultimately, the article encourages founders to remain committed and resilient in the face of challenges.'}

In [25]:
question2 = """
And how does it explain how to create somethin people want?
"""

In [26]:
# QA_chain.run(question2)
QA_chain.invoke(question2)

{'query': '\nAnd how does it explain how to create somethin people want?\n',
 'result': 'The essay suggests that to create something people want, you should focus on making something that at least a few users really love. This involves understanding what aspects of your product resonate with those users and expanding on that. It emphasizes the importance of launching quickly to gather user feedback, which can guide you in refining your product. The idea is to keep trying and tweaking your product based on user responses until you find a winning iteration. Additionally, having a core group of enthusiastic users can provide motivation and direction for your startup.'}

## Connect RAG App with FastAPI

In [27]:
#!pip install fastapi

In [28]:
from fastapi import FastAPI, HTTPException

In [29]:
app = FastAPI()

In [30]:
@app.post("/conversation")
async def conversation(query: str):
    try:
        result = QA_chain.run(query=query)
        return {"response": result}
    except Exception as e:
        raise HTTPException(detail=str(e), status_code=500)

In [31]:
#!pip install uvicorn

In [32]:
import uvicorn

In [33]:
config = uvicorn.Config(app, host="0.0.0.0", port=5566)
server = uvicorn.Server(config)

In [34]:
await server.serve()

INFO:     Started server process [9900]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:5566 (Press CTRL+C to quit)


INFO:     127.0.0.1:11449 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:11449 - "GET /openapi.json HTTP/1.1" 200 OK


C:\Users\Pavel\AppData\Local\Temp\ipykernel_9900\454693123.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = QA_chain.run(query=query)


INFO:     127.0.0.1:11463 - "POST /conversation?query=What%20is%20this%20article%20about%3F%20%20Describe%20it%20in%20less%20than%2050%20words. HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [9900]


* Open http://0.0.0.0:5566/docs#
* In Windows, open http://localhost:5566/docs#

### From Python file (add this code and execute python filename.py in terminal)

In [35]:
# if __name__ == "__main__":
#     import uvicorn

#     uvicorn.run(app, host="0.0.0.0", port=5566)